# Config

In [2]:
#editamos el path para poder accedera las funciones con la estructura actual de carpetas
from sys import path
import os
path.append(os.path.realpath('../'))

In [3]:
#importamos las funciones que hemos creado nosotros
from utils import functions as func

In [4]:
#importamos librería
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder,MinMaxScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer

import folium
import plotly.express as px
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [5]:
# Leemos los ficheros
ruta = '../data/'

df_calendar = pd.read_csv(ruta+'daily_calendar_with_events.csv')
df_sales = pd.read_csv(ruta+'item_sales.csv')
df_prices = pd.read_csv(ruta+'item_prices.csv')

# Datasets info

## Calendar

In [6]:
df_calendar.head()

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN


In [7]:
df_calendar.value_counts()

date        weekday    weekday_int  d       event         
2011-02-06  Sunday     2            d_9     SuperBowl         1
2011-08-01  Monday     3            d_185   Ramadan starts    1
2016-02-07  Sunday     2            d_1836  SuperBowl         1
2016-01-01  Friday     7            d_1799  NewYear           1
2015-11-26  Thursday   6            d_1763  Thanksgiving      1
2015-06-18  Thursday   6            d_1602  Ramadan starts    1
2015-04-05  Sunday     2            d_1528  Easter            1
2015-02-01  Sunday     2            d_1465  SuperBowl         1
2015-01-01  Thursday   6            d_1434  NewYear           1
2014-11-27  Thursday   6            d_1399  Thanksgiving      1
2014-06-29  Sunday     2            d_1248  Ramadan starts    1
2014-04-20  Sunday     2            d_1178  Easter            1
2014-02-02  Sunday     2            d_1101  SuperBowl         1
2014-01-01  Wednesday  5            d_1069  NewYear           1
2013-11-28  Thursday   6            d_1035  T

In [8]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         1913 non-null   object
 1   weekday      1913 non-null   object
 2   weekday_int  1913 non-null   int64 
 3   d            1913 non-null   object
 4   event        26 non-null     object
dtypes: int64(1), object(4)
memory usage: 74.9+ KB


In [9]:
df_calendar.head()

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN


In [10]:
# Prueba
#df_calendar.groupby(['event','event2','event3']).agg(total = ('d', 'count'))

In [11]:
df_calendar['holiday'] = df_calendar['event'].fillna(0).map(lambda x: 1 if x != 0 else 0)

In [12]:
df_calendar['date'] = pd.to_datetime(df_calendar['date'], errors='coerce')

In [13]:
# Extract the day of the week (Monday=0, Sunday=6)
df_calendar['weekday_int'] = df_calendar['date'].dt.weekday

# Create a new column 'weekend' based on the day of the week
df_calendar['weekend'] = df_calendar['weekday_int'].isin([5, 6]).astype(int)  # 5 and 6 correspond to Saturday and Sunday

In [14]:
df_calendar["yearweek"]= df_calendar['date'].apply(func.calcular_yearweek)

In [15]:
print('min date:' ,df_calendar['date'].min())
print('max date:' ,df_calendar['date'].max())
print('min yearweek:' ,df_calendar['yearweek'].min())
print('max yearweek:' ,df_calendar['yearweek'].max())

min date: 2011-01-29 00:00:00
max date: 2016-04-24 00:00:00
min yearweek: 201104
max yearweek: 201616


In [16]:
df_yearweek = pd.DataFrame({'yearweek': df_calendar['yearweek'].drop_duplicates(),
                       'key': 1})

## Sales

In [17]:
df_sales.head()

,id,item,category,department,store,store_code,region,d_1,d_2,d_3,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [18]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1920 entries, id to d_1913
dtypes: int64(1913), object(7)
memory usage: 446.6+ MB


In [19]:
#obtenemos un listado de los diferentes ids
df_ids = pd.DataFrame({'id': df_sales['id'].drop_duplicates(),
                       'key': 1})

In [20]:
df_ids.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30490 entries, 0 to 30489
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      30490 non-null  object
 1   key     30490 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 714.6+ KB


Pasamos las columnas dia a fila

In [21]:
#Pasamos las columas d a filas par poder trabajar con ellas
# Cogemos las columnas.
columnas = df_sales.columns
list_id_vars = columnas[:7]   # son las columnas que necesitamos mantener.
list_value_vars = columnas[7:]

# Codigo para trasponer columnas a filas.
df_daily_sales = pd.melt(df_sales, id_vars=list_id_vars,
                   value_vars=list_value_vars ,
                   var_name='d', value_name='qty_sold')

In [22]:
df_daily_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 9 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   id          object
 1   item        object
 2   category    object
 3   department  object
 4   store       object
 5   store_code  object
 6   region      object
 7   d           object
 8   qty_sold    int64 
dtypes: int64(1), object(8)
memory usage: 3.9+ GB


In [23]:
df_daily_sales["d"].describe()

count     58327370
unique        1913
top            d_1
freq         30490
Name: d, dtype: object

## Prices

In [24]:
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6965706 entries, 0 to 6965705
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   item        object 
 1   category    object 
 2   store_code  object 
 3   yearweek    float64
 4   sell_price  float64
dtypes: float64(2), object(3)
memory usage: 265.7+ MB


In [25]:
df_prices.head()

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.0,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,12.7414
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330.0,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331.0,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332.0,10.9858


In [26]:
df_prices.isnull().sum()

item               0
category           0
store_code         0
yearweek      243920
sell_price         0
dtype: int64

In [27]:
df_prices=df_prices.dropna(subset=["yearweek"])
df_prices.isnull().sum()

item          0
category      0
store_code    0
yearweek      0
sell_price    0
dtype: int64

In [28]:
# Pasamos la variable yearweek a modo fecha:
df_prices["yearweek"] = df_prices["yearweek"].astype("int64")

In [29]:
print('min yearweek:' ,df_prices['yearweek'].min())
print('max yearweek:' ,df_prices['yearweek'].max())

min yearweek: 201105
max yearweek: 201617


In [30]:
# Para hacer merge con el df_prices creamos columna d que es item+store_code dentro de prices
df_prices["id"]= df_prices["item"]+"_"+df_prices['store_code']

In [31]:
df_prices.head()

,item,category,store_code,yearweek,sell_price,id
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328,12.7414,ACCESORIES_1_001_NYC_1
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329,12.7414,ACCESORIES_1_001_NYC_1
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330,10.9858,ACCESORIES_1_001_NYC_1
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331,10.9858,ACCESORIES_1_001_NYC_1
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332,10.9858,ACCESORIES_1_001_NYC_1


In [32]:
del df_prices["item"], df_prices['store_code'], df_prices['category']

### Imputacion nulos en sellprice

In [33]:
#generamos las combinaciones posibles de semana y id, para luego generar las observaciones con nulos 
combinaciones_df = pd.merge(df_yearweek, df_ids, on='key').drop('key', axis=1)

In [34]:
combinaciones_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8354260 entries, 0 to 8354259
Data columns (total 2 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   yearweek  int64 
 1   id        object
dtypes: int64(1), object(1)
memory usage: 127.5+ MB


In [35]:
print('min yearweek:' ,combinaciones_df['yearweek'].min())
print('max yearweek:' ,combinaciones_df['yearweek'].max())

min yearweek: 201104
max yearweek: 201616


In [36]:
df_prices = pd.merge(combinaciones_df, df_prices, on=['id', 'yearweek'], how='left')

In [37]:
df_prices.sort_values(['id', 'yearweek'], ascending = True, inplace = True)

In [38]:
print('min yearweek:' ,df_prices['yearweek'].min())
print('max yearweek:' ,df_prices['yearweek'].max())

min yearweek: 201104
max yearweek: 201616


In [39]:
#df_prices[df_prices['sell_price'].isnull()].sum()

In [40]:
df_prices[df_prices['yearweek']==201553]

,yearweek,id,sell_price
7848126,201553,ACCESORIES_1_001_BOS_1,NaN
7851175,201553,ACCESORIES_1_001_BOS_2,NaN
7854224,201553,ACCESORIES_1_001_BOS_3,NaN
7835930,201553,ACCESORIES_1_001_NYC_1,NaN
7838979,201553,ACCESORIES_1_001_NYC_2,NaN
...,...,...,...
7845076,201553,SUPERMARKET_3_827_NYC_3,NaN
7848125,201553,SUPERMARKET_3_827_NYC_4,NaN
7860321,201553,SUPERMARKET_3_827_PHI_1,NaN
7863370,201553,SUPERMARKET_3_827_PHI_2,NaN


Para ver si hacemos bien la imputacion seleccionamos algun id con nulos para comprobar

In [41]:
rows_with_null_sell_price = df_prices[df_prices['sell_price'].isnull()]

# Seleccionar un ID que tenga al menos una semana con 'sell_price' nulo
selected_id = rows_with_null_sell_price['id'].sample(n=1).iloc[0]

# Mostrar el ID seleccionado
print(f"ID seleccionado: {selected_id}")

ID seleccionado: ACCESORIES_1_003_BOS_3


In [42]:
sample_ts = df_prices[df_prices['id']=='ACCESORIES_1_001_BOS_2']

In [43]:
df_prices["sell_price"] = df_prices.groupby("id", group_keys = False)["sell_price"].apply(
    lambda series: series.backfill().ffill())

In [44]:
df_prices[df_prices['yearweek']==201553]

,yearweek,id,sell_price
7848126,201553,ACCESORIES_1_001_BOS_1,10.9858
7851175,201553,ACCESORIES_1_001_BOS_2,10.9858
7854224,201553,ACCESORIES_1_001_BOS_3,10.9858
7835930,201553,ACCESORIES_1_001_NYC_1,10.9858
7838979,201553,ACCESORIES_1_001_NYC_2,10.9858
...,...,...,...
7845076,201553,SUPERMARKET_3_827_NYC_3,1.2000
7848125,201553,SUPERMARKET_3_827_NYC_4,1.2000
7860321,201553,SUPERMARKET_3_827_PHI_1,1.2000
7863370,201553,SUPERMARKET_3_827_PHI_2,1.2000


# Union datasets

## Calendar sales 
Realizamos la union del df_calendar y el df_daily sales a través de la variable d.

In [45]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         1913 non-null   datetime64[ns]
 1   weekday      1913 non-null   object        
 2   weekday_int  1913 non-null   int32         
 3   d            1913 non-null   object        
 4   event        26 non-null     object        
 5   holiday      1913 non-null   int64         
 6   weekend      1913 non-null   int32         
 7   yearweek     1913 non-null   int64         
dtypes: datetime64[ns](1), int32(2), int64(2), object(3)
memory usage: 104.7+ KB


In [46]:
df_calendar.head()

,date,weekday,weekday_int,d,event,holiday,weekend,yearweek
0,2011-01-29,Saturday,5,d_1,NaN,0,1,201104
1,2011-01-30,Sunday,6,d_2,NaN,0,1,201104
2,2011-01-31,Monday,0,d_3,NaN,0,0,201105
3,2011-02-01,Tuesday,1,d_4,NaN,0,0,201105
4,2011-02-02,Wednesday,2,d_5,NaN,0,0,201105


In [47]:
df_calendar_sales = pd.merge(df_daily_sales,df_calendar[['date','d','weekday_int','holiday','weekend','yearweek']], on='d', how='inner')

In [48]:
df_calendar_sales.head()

,id,item,category,department,store,store_code,region,d,qty_sold,date,weekday_int,holiday,weekend,yearweek
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,5,0,1,201104
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,5,0,1,201104
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,5,0,1,201104
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,5,0,1,201104
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,5,0,1,201104


In [49]:
df_calendar_sales.shape

(58327370, 14)

In [50]:
print('min date:' ,df_calendar_sales['date'].min())
print('max date:' ,df_calendar_sales['date'].max())
print('min yearweek:' ,df_calendar_sales['yearweek'].min())
print('max yearweek:' ,df_calendar_sales['yearweek'].max())

min date: 2011-01-29 00:00:00
max date: 2016-04-24 00:00:00
min yearweek: 201104
max yearweek: 201616


## Reducimos dimensionalidad

In [51]:
df_calendar_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 14 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           object        
 1   item         object        
 2   category     object        
 3   department   object        
 4   store        object        
 5   store_code   object        
 6   region       object        
 7   d            object        
 8   qty_sold     int64         
 9   date         datetime64[ns]
 10  weekday_int  int32         
 11  holiday      int64         
 12  weekend      int32         
 13  yearweek     int64         
dtypes: datetime64[ns](1), int32(2), int64(3), object(8)
memory usage: 5.6+ GB


In [52]:
df_weekly_sales = (df_calendar_sales
                   .groupby(['id', 'item', 'category', 'department', 'store', 'store_code', 'region', 'yearweek'], as_index=False)
                   .agg(
                        qty_sold = ('qty_sold', 'sum'),
                        date=('date', 'min'),
                        holiday=('holiday', 'max')
                        
                    )
                   
                   )


- pd.Grouper es una clase en la biblioteca pandas que proporciona una interfaz flexible para realizar operaciones de agrupación en series temporales. Se utiliza comúnmente con el método groupby para agrupar datos temporales de manera conveniente y realizar operaciones de agregación en esos grupos.
- key='date': Especifica la columna por la cual se realizará la agrupación. En este caso, la columna 'date'.
- freq='W-Sun': Especifica la frecuencia con la que se agruparán los datos. En este ejemplo, se agruparán semanalmente ('W') y el doming ('sun') será el día de inicio de la semana.

In [ ]:
df_weekly_sales.shape

(8354260, 11)

In [ ]:
df_weekly_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8354260 entries, 0 to 8354259
Data columns (total 11 columns):
 #   Column      Dtype         
---  ------      -----         
 0   id          object        
 1   item        object        
 2   category    object        
 3   department  object        
 4   store       object        
 5   store_code  object        
 6   region      object        
 7   yearweek    int64         
 8   qty_sold    int64         
 9   date        datetime64[ns]
 10  holiday     int64         
dtypes: datetime64[ns](1), int64(3), object(7)
memory usage: 701.1+ MB


In [ ]:
print('min yearweek:' ,df_weekly_sales['yearweek'].min())
print('max yearweek:' ,df_weekly_sales['yearweek'].max())

min yearweek: 201104
max yearweek: 201616


## Union Sales con Prices

In [ ]:
df_weekly_sales_prices = pd.merge(df_weekly_sales, df_prices, on=['id', "yearweek"], how='left')  

In [ ]:
df_calendar_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 14 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           object        
 1   item         object        
 2   category     object        
 3   department   object        
 4   store        object        
 5   store_code   object        
 6   region       object        
 7   d            object        
 8   qty_sold     int64         
 9   date         datetime64[ns]
 10  weekday_int  int32         
 11  holiday      int64         
 12  weekend      int32         
 13  yearweek     int64         
dtypes: datetime64[ns](1), int32(2), int64(3), object(8)
memory usage: 5.6+ GB


In [ ]:
df_daily_sales_prices = pd.merge(df_calendar_sales, df_prices, on=['id', "yearweek"], how='left')  

In [ ]:
print('min yearweek:' ,df_weekly_sales_prices['yearweek'].min())
print('max yearweek:' ,df_weekly_sales_prices['yearweek'].max())

min yearweek: 201104
max yearweek: 201616


In [ ]:
df_weekly_sales_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8354260 entries, 0 to 8354259
Data columns (total 12 columns):
 #   Column      Dtype         
---  ------      -----         
 0   id          object        
 1   item        object        
 2   category    object        
 3   department  object        
 4   store       object        
 5   store_code  object        
 6   region      object        
 7   yearweek    int64         
 8   qty_sold    int64         
 9   date        datetime64[ns]
 10  holiday     int64         
 11  sell_price  float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(7)
memory usage: 764.9+ MB


# Export Data

In [ ]:
#df_daily_sales_prices.to_csv((ruta+'daily_sales_cleandata.csv'), index=False)

In [ ]:
df_weekly_sales_prices.to_csv((ruta+'weekly_sales_cleandata.csv'), index=False)

In [ ]:
#df_weekly_sales_prices.info()

NameError: name 'df_weekly_sales_prices' is not defined